In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import time
import copy
import string
import json
import pickle as pk

from treelib import Node, Tree
from math import sin, asin, cos, radians, fabs, sqrt

In [2]:
############################    从文件中 读取 数据   ##############
def read_q(filename):
    f = open(filename,"rb+")
    st = pk.load(f)
    f.close()
    return st

In [3]:
##########################    向文件中  写 数据  #####################
def write_q(st, filename):
    f = open(filename, "wb+")
    pk.dump(st,f)
    f.close()

In [5]:
###################  match()  停留点与语义树叶子结点的匹配  ##############################
# 输入 si_h_f[i]   一维list ; st_l   语义树的所有叶子结点
# 输出 sit_l    匹配后的叶子结点

#si = ['北京郊野园中园', 39.996209, 116.26688600000001, 13.75000013387762, 'farm', 1]
#st_l = st.leaves()
#n_id = match(si, st_l)
#st.get_node(n_id)

def match(si, st_l):
    for i in range(len(st_l)):
    
        if st_l[i].data == None:
            continue
        else:
        
            if si[0]==st_l[i].data["id"] and si[1]==st_l[i].data["location"]["lat"] and \
            si[2]==st_l[i].data["location"]["lng"]:
                n_id = st_l[i].identifier
                break

            
    return n_id       

In [6]:
from math import radians, cos, sin, asin, sqrt
def distance_vec(lon1, lat1, lon2_v, lat2_v): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)   """
    # 将十进制度数转化为弧度
    lon1, lat1 = map(radians, [lon1, lat1])
    lon2_v=np.radians(lon2_v)
    lat2_v=np.radians(lat2_v)
    dlon = lon2_v - lon1 
    dlat = lat2_v - lat1 
    a = np.power(np.sin(dlat/2),2) + cos(lat1) *np.cos(lat2_v) *  np.power(np.sin(dlon/2),2)#**2
    c = 2 *np.arcsin(np.sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    dis_l=c * r * 1000
    #print(dis_l.argmin())
    return dis_l

In [15]:
##########################    生成K-匿名集  ##################
## 输入 POI si;   候选匿名集 m_l;  k
## 输出 K-匿名集     k_as   [{},{},{},{}]

#si = ['北京郊野园中园', 39.996209, 116.26688600000001, 13.75000013387762, 'farm', 1]
#m_l = st.leaves("farm")
#k = 3
#a = gen_k_as(si, m_l, k)
#a

def gen_k_as(si, m_l, k):
    
    k_as = []
    #dis = []
    m_l2 = copy.deepcopy(m_l)
    
    print("K-匿名集生成前叶子结点list长度： ",len(m_l2))
    
    ##########  删除空结点  ##########
    j = 0
    while j < len(m_l2):     #  删除空叶子结点
        
        if m_l2[j].data == None:
            del m_l2[j]
        else:
            j += 1
            
    print("K-匿名集生成后叶子结点list长度： ",len(m_l2))    
    
    ####  距离函数重写    #######
    lat = si[1]
    lng = si[2]
    
    m_l2_loc = []
    for i in range(len(m_l2)):
        n_d = m_l2[i].data
        a = []
        a = [n_d["location"]["lat"],n_d["location"]["lng"]]
        m_l2_loc.append(a)
        
    df_ml_list=pd.DataFrame(m_l2_loc,columns=['la','lo'])
    la_l=np.array(df_ml_list['la'])
    lo_l=np.array(df_ml_list['lo'])
    
    dis_l = distance_vec(lng, lat, lo_l, la_l)
    #print(dis_l)
    
    for l in range(len(m_l2)):    # 添加距离字段
        m_l2[l].data["distance"] = dis_l[l]
        
    ##########
            
    print("计算距离后，打印第一个叶子结点： ", m_l2[0])
    m = sorted(m_l2,key = distance)      #自定义函数按列表f中字典的distance从小到大排序， 返回结点 list    
    
    if len(m) < k:
        k_as = m
    
    else:
        for j in range(k):
            k_as.append(m[j])
            
    return k_as   

def distance(s): 
    return s.data['distance'] 

In [16]:
si = ['北京大学附属小学-体育馆', 40.000085, 116.32512, 105.74999999371357, 'bbulid', 3]
m_l = st.leaves("bbulid")
k = 3
a = gen_k_as(si, m_l, k)
a

K-匿名集生成前叶子结点list长度：  359
K-匿名集生成后叶子结点list长度：  356
计算距离后，打印第一个叶子结点：  Node(tag=enter_bbulid_0_0_0, identifier=enter_bbulid_0_0_0, data={'id': '马甸公园-篮球场', 'location': {'lat': 39.975856, 'lng': 116.38586299999999}, 'staying': [10.666666656034067], 'category': 'bbulid', 'distance': 5834.31106325196})


[Node(tag=enter_bbulid_2_2_0, identifier=enter_bbulid_2_2_0, data={'id': '北京大学附属小学-体育馆', 'location': {'lat': 40.000085, 'lng': 116.32512}, 'staying': [212.31666662497446, 180.0, 21.683333372930065, 99.79999992297962, 172.9333332006354, 27.00000000419095, 196.2333332723938, 82.84999997355044, 50.7666666212026, 45.36666676285677, 41.433333264430985, 182.74999997229315, 333.04999996558763, 143.0833334417548, 212.31666662497446, 180.0, 21.683333372930065, 99.79999992297962, 172.9333332006354, 27.00000000419095, 196.2333332723938, 82.84999997355044, 50.7666666212026, 45.36666676285677, 132.18333335593343, 306.6833332856186, 126.46666670450941, 144.58333334187046, 13.250000016996637, 222.03333331039175, 221.56666660797782, 67.59999992907979, 94.09999996307306, 78.2500000402797, 156.9833333278075, 215.98333328962326, 340.3666666033678, 106.3333333376795, 179.4333332113456, 122.26666665519588, 357.1833333559334, 78.83333338424563, 273.41666669002734, 99.51666667009704, 97.60000003734604, 48.41

In [ ]:
si

In [8]:
i = 0
j = 47
filename = ".\\os_file\\si_h_f_"+str(i)+"_"+str(j)+".pkl"
si_h_f = read_q(filename)
si_h_f

[['无尽', 40.005973, 116.32533400000001, 65.81666663987562, 'scenic', 2],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 19.833333265269175, 'scenic', 2],
 ['北京大学附属小学-体育馆', 40.000085, 116.32512, 105.74999999371357, 'bbulid', 3],
 ['格林豪泰酒店(首都机场航站楼店)', 40.082075, 116.59334, 10.333333286689594, 'ehotel', 2],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 16.333333337679505, 'scenic', 2],
 ['绮春园-舞台', 40.010327000000004, 116.314555, 22.5, 'scenic', 2],
 ['万春园丙区', 40.007911, 116.31916399999999, 10.3333334333729, 'scenic', 2],
 ['生冬室', 40.009062, 116.315025, 342.0833333779592, 'scenic', 2],
 ['生冬室', 40.009062, 116.315025, 136.0833333141636, 'scenic', 2],
 ['生冬室', 40.009062, 116.315025, 407.08333325455897, 'scenic', 2],
 ['颐和园-治镜阁遗址',
  39.997733000000004,
  116.270547,
  12.166666692355648,
  'scenic',
  2],
 ['北京郊野园中园', 39.996209, 116.26688600000001, 13.75000013387762, 'farm', 1],
 ['宏都商场', 39.908799, 116.40641000000001, 38.08333324617706, 'smarket', 1],
 ['天安门广场', 39.909652, 116.404177, 11.66666659

In [9]:
filename = ".\\os_file\\semantic_tree.pkl"
st = read_q(filename)
st.show()

POI
├── Enter
│   ├── Bar
│   │   ├── enter_bar_0
│   │   │   ├── enter_bar_0_0
│   │   │   │   ├── enter_bar_0_0_0
│   │   │   │   ├── enter_bar_0_0_1
│   │   │   │   ├── enter_bar_0_0_10
│   │   │   │   ├── enter_bar_0_0_11
│   │   │   │   ├── enter_bar_0_0_12
│   │   │   │   ├── enter_bar_0_0_13
│   │   │   │   ├── enter_bar_0_0_14
│   │   │   │   ├── enter_bar_0_0_15
│   │   │   │   ├── enter_bar_0_0_2
│   │   │   │   ├── enter_bar_0_0_3
│   │   │   │   ├── enter_bar_0_0_4
│   │   │   │   ├── enter_bar_0_0_5
│   │   │   │   ├── enter_bar_0_0_6
│   │   │   │   ├── enter_bar_0_0_7
│   │   │   │   ├── enter_bar_0_0_8
│   │   │   │   └── enter_bar_0_0_9
│   │   │   ├── enter_bar_0_1
│   │   │   │   └── enter_bar_0_1_0
│   │   │   ├── enter_bar_0_2
│   │   │   │   └── enter_bar_0_2_0
│   │   │   └── enter_bar_0_3
│   │   ├── enter_bar_1
│   │   │   ├── enter_bar_1_0
│   │   │   ├── enter_bar_1_1
│   │   │   ├── enter_bar_1_2
│   │   │   ├── enter_bar_1_3
│   │   │   ├── enter_bar_1_4
│ 

In [19]:
#####################  轨迹重构 rec_tra  rec_tra_i_j.pkl ################
# 输入  si_h_f 文件内停留点的语义高度； st 语义树；
# 重构轨迹 rec_tra  
s = 0     # 用户
t = 47    # 文件

filename = ".\\os_file\\rec_tra_"+str(s)+"_"+str(t)+".pkl" 

k = 4

si = si_h_f    # 文件中停留点list 6元组

st_l = st.leaves()   # st 叶子结点list

rec_tra = []

for i in range(len(si)):
    
    h = si[i][5]      # 语义匿名高度
    n_id = match(si[i], st_l)     # 文件停留点与叶子结点匹配，   返回匹配结点  id
    print(n_id)
    m_id = n_id
    if st.level(m_id)+1 < h:
        m_id = "poi"             #如果回溯的层次大于它的高度，则直接以跟结点为中间结点
    else:
        for j in range(h):       # 否则回溯
            m_id = st.parent(m_id).identifier   # 返回 中间结点 id
    
    print(m_id)
    m_l = st.leaves(m_id)    # 返回 中间结点 对应的 叶子结点  匿名集
    print(len(m_l))
    k_as = gen_k_as(si[i], m_l, k)       # 生成K-匿名集，  [node,node,node]    在中间结点对应的叶子结点中，基于距离生成K-匿名集
    
    idx = np.random.randint(k)      # k范围内，随机生成索引
    
    rec_tra.append(k_as[idx])
    
write_q(rec_tra, filename)
rec_tra

enter_scenic_0_1_0
enter_scenic_0
62
K-匿名集生成前叶子结点list长度：  62
K-匿名集生成后叶子结点list长度：  62
计算距离后，打印第一个叶子结点：  Node(tag=enter_scenic_0_0_0, identifier=enter_scenic_0_0_0, data={'id': '岂骒错登闻身溪', 'location': {'lat': 39.884492, 'lng': 116.328073}, 'staying': [91.38333340524696], 'category': 'scenic', 'distance': 13510.088793136276})
enter_scenic_2_2_9
enter_scenic_2
149
K-匿名集生成前叶子结点list长度：  149
K-匿名集生成后叶子结点list长度：  149
计算距离后，打印第一个叶子结点：  Node(tag=enter_scenic_2_0_0, identifier=enter_scenic_2_0_0, data={'id': '人保博物馆', 'location': {'lat': 40.007597, 'lng': 116.320035}, 'staying': [59.0666667802725, 44.59999996935949, 29.71666669822298, 11.666666732635349, 10.083333312068135, 59.0666667802725, 44.59999996935949, 29.71666669822298, 329.11666661384515, 132.5666666845791, 61.150000034831464, 11.333333373768255, 37.00000008917414, 245.71666670031846, 34.749999940395355, 114.09999997587875, 18.383333324454725, 45.883333288365975, 83.3333333779592, 14.083333199378103, 161.3999999477528, 183.86666667996906,

[Node(tag=enter_scenic_0_1_21, identifier=enter_scenic_0_1_21, data={'id': '零零阁', 'location': {'lat': 40.006861, 'lng': 116.326044}, 'staying': [88.23333340231329], 'category': 'scenic', 'distance': 115.78731618244935}),
 Node(tag=enter_scenic_2_2_9, identifier=enter_scenic_2_2_9, data={'id': '绮春园-舞台', 'location': {'lat': 40.010327000000004, 'lng': 116.314555}, 'staying': [93.01666680607013, 253.9333333389368, 83.34999993327074, 25.850000018253922, 12.499999915016815, 19.833333265269175, 16.333333337679505, 22.5, 10.599999984260648, 54.36666663503274, 10.333333297166973, 29.666666728444397, 13.450000032316893, 42.399999937042594, 82.91666664532386, 81.49999996181577, 72.21666672150604, 33.583333399146795, 12.499999915016815, 19.833333265269175, 16.333333337679505, 22.5, 10.599999984260648, 54.36666663503274, 10.333333297166973, 29.666666728444397, 13.450000032316893, 42.399999937042594, 82.91666664532386, 66.11666659475304, 342.04999996349216, 311.63333341595717, 63.483333410695195, 35

In [20]:
len(rec_tra)

14

In [4]:
filename = ".\\os_file\\rec_tra_0_47.pkl"
rec = read_q(filename)
rec

[Node(tag=enter_scenic_0_1_21, identifier=enter_scenic_0_1_21, data={'id': '零零阁', 'location': {'lat': 40.006861, 'lng': 116.326044}, 'staying': [88.23333340231329], 'category': 'scenic', 'distance': 115.78731618244935}),
 Node(tag=enter_scenic_2_2_9, identifier=enter_scenic_2_2_9, data={'id': '绮春园-舞台', 'location': {'lat': 40.010327000000004, 'lng': 116.314555}, 'staying': [93.01666680607013, 253.9333333389368, 83.34999993327074, 25.850000018253922, 12.499999915016815, 19.833333265269175, 16.333333337679505, 22.5, 10.599999984260648, 54.36666663503274, 10.333333297166973, 29.666666728444397, 13.450000032316893, 42.399999937042594, 82.91666664532386, 81.49999996181577, 72.21666672150604, 33.583333399146795, 12.499999915016815, 19.833333265269175, 16.333333337679505, 22.5, 10.599999984260648, 54.36666663503274, 10.333333297166973, 29.666666728444397, 13.450000032316893, 42.399999937042594, 82.91666664532386, 66.11666659475304, 342.04999996349216, 311.63333341595717, 63.483333410695195, 35